In [1]:
import sys
import pandas as pd
import requests

def get_publication_year_crossref(doi_link):
    if pd.isna(doi_link) or "doi.org" not in doi_link:
        return None
    api_url = f"https://api.crossref.org/works/{doi_link.split('doi.org/')[1]}"
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            if 'message' in data and 'published-print' in data['message']:
                year = data['message']['published-print']['date-parts'][0][0]
                return str(year)
            elif 'message' in data and 'published-online' in data['message']:
                year = data['message']['published-online']['date-parts'][0][0]
                return str(year)
        return None
    except Exception as e:
        return None

# Step 4: Load the Excel file
file_path = 'TE_Data_V1.xlsx'  # Replace with your file path
df = pd.read_excel(file_path)

# Step 5: Apply the function to the DOI column
df['Publication Year'] = df['DOI'].apply(get_publication_year_crossref)

# Step 6: Save the updated DataFrame back to Excel
output_file_path = 'updated_file.xlsx'  # Replace with your desired output file path
df.to_excel(output_file_path, index=False)

# Confirmation message
print(f"Updated Excel file saved as {output_file_path}")

KeyboardInterrupt: 

In [1]:
# Step 1: Install Necessary Libraries
import sys

# Step 2: Import Libraries
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

# Step 3: Determine maximum threads based on system CPU count
def get_max_threads():
    # Get the number of CPU cores
    cpu_count = multiprocessing.cpu_count()
    # For I/O-bound tasks, it's often safe to use more threads than CPU cores
    max_threads = cpu_count * 2  # Adjust multiplier based on task needs
    return max_threads

# Step 4: Define the function to get the publication year using CrossRef API
def get_publication_year_crossref(doi_link):
    if pd.isna(doi_link) or "doi.org" not in doi_link:
        return None
    api_url = f"https://api.crossref.org/works/{doi_link.split('doi.org/')[1]}"
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            if 'message' in data and 'published-print' in data['message']:
                year = data['message']['published-print']['date-parts'][0][0]
                return str(year)
            elif 'message' in data and 'published-online' in data['message']:
                year = data['message']['published-online']['date-parts'][0][0]
                return str(year)
        return None
    except Exception as e:
        return None

# Step 5: Load the Excel file
file_path = 'TE_Data_V1.xlsx'  # Replace with your actual file path
df = pd.read_excel(file_path)
df=df.head(10) # Limiting to first 10 rows for demonstration purposes

# Step 6: Remove duplicates and create a lookup dictionary for DOIs in the full dataset
unique_dois = df['DOI'].dropna().unique()

# Step 7: Use ThreadPoolExecutor with maximum workers based on system capacity
max_workers = get_max_threads()
print(f"Using {max_workers} threads based on system capacity.")  # Optional: Print the number of threads being used

doi_to_year = {}
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_doi = {executor.submit(get_publication_year_crossref, doi): doi for doi in unique_dois}
    for future in as_completed(future_to_doi):
        doi = future_to_doi[future]
        try:
            year = future.result()
            doi_to_year[doi] = year
        except Exception as e:
            doi_to_year[doi] = None

# Step 8: Map the results back to the full DataFrame
df['Publication Year'] = df['DOI'].map(doi_to_year)

# Step 9: Display the updated full DataFrame
print(df.head())  # Display the first few rows to verify

# Step 10: Save the full updated DataFrame back to Excel
output_file_path = 'updated_full_dataset.xlsx'  # Replace with your desired output file path
df.to_excel(output_file_path, index=False)

# Confirmation message
print(f"Full dataset saved as {output_file_path}")

Defaulting to user installation because normal site-packages is not writeable
Using 20 threads based on system capacity.
     # Source                                 DOI    Pretty Formula  \
0  0.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
1  1.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
2  2.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
3  3.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
4  4.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   

  Full Formula                   Series Family Host Material Dopant Material  \
0          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
1          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
2          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
3          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
4          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     P

In [13]:
# Step 2: Import Libraries
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing
import time
import logging
import random
import re

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Track failed DOIs for manual inspection
failed_dois = []

# Step 3: Determine maximum threads based on system CPU count
def get_max_threads():
    cpu_count = multiprocessing.cpu_count()
    max_threads = max(1, cpu_count // 2)  # Reduce threads to avoid API rate limits
    return max_threads

# Step 4: Define the function to extract DOI suffix and get the publication year using CrossRef API with enhanced retries
def get_publication_year_crossref(doi_link):
    if pd.isna(doi_link):
        logging.error(f"Invalid DOI format: {doi_link}")
        failed_dois.append(doi_link)
        return None
    
    # Extract DOI suffix using regex to handle different formats
    match = re.search(r'(10\.\d{4,9}/[-._;()/:A-Z0-9]+)', doi_link, re.IGNORECASE)
    if not match:
        logging.error(f"Could not extract DOI from: {doi_link}")
        failed_dois.append(doi_link)
        return None

    doi_suffix = match.group(1)  # Get the DOI part of the link
    api_url = f"https://api.crossref.org/works/{doi_suffix}"
    
    retries = 10
    last_status_code = None
    for attempt in range(retries):
        try:
            response = requests.get(api_url)
            last_status_code = response.status_code
            if response.status_code == 200:
                data = response.json()
                # Log the entire response to understand its structure if extraction fails
                if 'message' in data:
                    message = data['message']
                    if 'published-print' in message and 'date-parts' in message['published-print']:
                        year = message['published-print']['date-parts'][0][0]
                        return str(year)
                    elif 'published-online' in message and 'date-parts' in message['published-online']:
                        year = message['published-online']['date-parts'][0][0]
                        return str(year)
                    else:
                        # Log detailed information about missing publication year
                        logging.error(f"Publication year not found in response for DOI: {doi_link}. Response: {data}")
                else:
                    logging.error(f"Unexpected response format for DOI: {doi_link}. Response: {data}")
                # Break early if we identify the response structure issue to avoid unnecessary retries
                break
            elif response.status_code == 404:
                logging.error(f"DOI not found in CrossRef: {doi_link}")
                failed_dois.append(doi_link)
                return None
            elif response.status_code == 429:
                retry_after = response.headers.get("Retry-After")
                if retry_after:
                    sleep_time = int(retry_after) + random.uniform(1, 5)
                    logging.warning(f"Rate limit hit, retrying after {sleep_time} seconds for DOI: {doi_link}")
                    time.sleep(sleep_time)
                else:
                    sleep_time = 180
                    logging.warning(f"Retry {attempt + 1}/{retries} for DOI: {doi_link} due to {response.status_code}, pausing for {sleep_time} seconds")
                    time.sleep(sleep_time)
            elif response.status_code in {500, 503}:
                logging.warning(f"Retry {attempt + 1}/{retries} for DOI: {doi_link} due to {response.status_code}")
                time.sleep(2 ** attempt)
            else:
                logging.error(f"Failed to get year for DOI: {doi_link} with status code: {response.status_code}")
                failed_dois.append(doi_link)
                return None
        except Exception as e:
            logging.error(f"Exception for DOI: {doi_link}, attempt {attempt + 1}/{retries}: {e}")
            time.sleep(2 ** attempt)
    
    logging.error(f"Exceeded maximum retries for DOI: {doi_link}, last status code: {last_status_code}")
    failed_dois.append(doi_link)
    return None

# Step 5: Load the Excel file
file_path = 'TE_Data_V1.xlsx'  # Replace with your actual file path
df = pd.read_excel(file_path)

# Step 6: Remove duplicates and create a lookup dictionary for DOIs in the full dataset
unique_dois = df['DOI'].dropna().unique()

# Step 7: Use ThreadPoolExecutor with maximum workers based on system capacity
max_workers = get_max_threads()
print(f"Using {max_workers} threads based on system capacity.")

doi_to_year = {}
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_doi = {executor.submit(get_publication_year_crossref, doi): doi for doi in unique_dois}
    for future in as_completed(future_to_doi):
        doi = future_to_doi[future]
        try:
            year = future.result()
            doi_to_year[doi] = year
        except Exception as e:
            doi_to_year[doi] = None
            logging.error(f"Failed to get result for DOI: {doi} with error: {e}")

# Step 8: Map the results back to the full DataFrame
df['Publication Year'] = df['DOI'].map(doi_to_year)

# Step 9: Display the updated full DataFrame
print(df.head())

# Step 10: Save the full updated DataFrame back to Excel
output_file_path = 'updated_full_dataset.xlsx'  # Replace with your desired output file path
df.to_excel(output_file_path, index=False)

# Step 11: Log failed DOIs for further inspection
if failed_dois:
    with open('failed_dois.txt', 'w') as file:
        for doi in failed_dois:
            file.write(f"{doi}\n")
    print(f"Failed DOIs saved in 'failed_dois.txt' for manual inspection.")

# Confirmation message
print(f"Full dataset saved as {output_file_path}")

Using 5 threads based on system capacity.


2024-08-30 16:56:56,343 - ERROR - Could not extract DOI from: http://www.jmst.org/EN/Y2009/V25/I04/0535
2024-08-30 16:56:57,379 - ERROR - Could not extract DOI from: Personal communication
2024-08-30 16:56:58,641 - ERROR - DOI not found in CrossRef: http://dx.doi.org/10.1016/j.jssc.2008.08.078
2024-08-30 16:56:58,773 - ERROR - Publication year not found in response for DOI: http://dx.doi.org/10.1109/ICT.1996.553263. Response: {'status': 'ok', 'message-type': 'work', 'message-version': '1.0.0', 'message': {'indexed': {'date-parts': [[2024, 1, 9]], 'date-time': '2024-01-09T00:03:21Z', 'timestamp': 1704758601886}, 'reference-count': 18, 'publisher': 'IEEE', 'content-domain': {'domain': [], 'crossmark-restriction': False}, 'short-container-title': [], 'DOI': '10.1109/ict.1996.553263', 'type': 'proceedings-article', 'created': {'date-parts': [[2002, 12, 24]], 'date-time': '2002-12-24T01:52:43Z', 'timestamp': 1040694763000}, 'source': 'Crossref', 'is-referenced-by-count': 59, 'title': ['High

     # Source                                 DOI    Pretty Formula  \
0  0.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
1  1.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
2  2.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
3  3.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   
4  4.0      1  https://doi.org/10.1039/D1EE02986J  Pb0.975Ga0.025Te   

  Full Formula                   Series Family Host Material Dopant Material  \
0          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
1          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
2          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
3          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   
4          NaN  Pb0.975Ga0.025Te–x%ZnTe   PbTe     Pb0.975Te         Ga0.025   

   Pure Form?  ... Formula Units per Cell Atoms per formula unit  \
0       False  ...      